<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n30_x4_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n30_x4_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    41.270721  19.234514  ... -34370.759259  1.681866e+06
     1.0    26.569659  19.200749  ... -43057.285551  1.687056e+05
     2.0    37.237705  43.553967  ...  11236.532193  2.782090e+06
     3.0    18.564953  38.405131  ...    635.126429  2.347863e+04
     4.0    22.397801  35.896585  ...  -6415.704600 -2.067549e+05
...               ...        ...  ...           ...           ...
99.0 25.0   40.536119  24.577691  ... -32687.532769  1.005290e+06
     26.0   31.556027  18.455317  ... -46221.387453 -2.299138e+05
     27.0   42.727903  40.809855  ...  11439.129215 -6.174272e+06
     28.0   16.792176  42.163700  ...   5949.228319  4.841186e+04
     29.0   20.750994  24.306281  ... -23322.762200 -7.336071e+04

[3000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    41.270721  19.234514  ... -34370.759259  1.681866e+06
     1.0    26.569659  19.200749  ... -43057.285551  1.687056e+05
     2.0    37.237705  43.553967  ...  11236.532193  2.782090e+06
     3.0    18.564953  38.405131  ...    635.126429  2.347863e+04
     4.0    22.397801  35.896585  ...  -6415.704600 -2.067549e+05
...               ...        ...  ...           ...           ...
99.0 25.0   40.536119  24.577691  ... -32687.532769  1.005290e+06
     26.0   31.556027  18.455317  ... -46221.387453 -2.299138e+05
     27.0   42.727903  40.809855  ...  11439.129215 -6.174272e+06
     28.0   16.792176  42.163700  ...   5949.228319  4.841186e+04
     29.0   20.750994  24.306281  ... -23322.762200 -7.336071e+04

[3000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,41.270721,19.234514,20.650985,1.647037e+06,1.647495e+06,-34370.759259,1.681866e+06
1,0.0,1.0,26.569659,19.200749,42.517434,1.253159e+05,1.256483e+05,-43057.285551,1.687056e+05
2,0.0,2.0,37.237705,43.553967,35.245638,2.792870e+06,2.793326e+06,11236.532193,2.782090e+06
3,0.0,3.0,18.564953,38.405131,35.191162,2.384951e+04,2.411375e+04,635.126429,2.347863e+04
4,0.0,4.0,22.397801,35.896585,39.378128,-2.134749e+05,-2.131706e+05,-6415.704600,-2.067549e+05
...,...,...,...,...,...,...,...,...,...
2995,99.0,25.0,40.536119,24.577691,44.508226,9.721233e+05,9.726027e+05,-32687.532769,1.005290e+06
2996,99.0,26.0,31.556027,18.455317,46.273464,-2.765205e+05,-2.761352e+05,-46221.387453,-2.299138e+05
2997,99.0,27.0,42.727903,40.809855,20.168064,-6.163327e+06,-6.162833e+06,11439.129215,-6.174272e+06
2998,99.0,28.0,16.792176,42.163700,41.784973,5.410422e+04,5.436109e+04,5949.228319,4.841186e+04


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    41.270721  19.234514  ... -34370.759259  1.681866e+06
     1.0    26.569659  19.200749  ... -43057.285551  1.687056e+05
     2.0    37.237705  43.553967  ...  11236.532193  2.782090e+06
     3.0    18.564953  38.405131  ...    635.126429  2.347863e+04
     4.0    22.397801  35.896585  ...  -6415.704600 -2.067549e+05
...               ...        ...  ...           ...           ...
99.0 25.0   40.536119  24.577691  ... -32687.532769  1.005290e+06
     26.0   31.556027  18.455317  ... -46221.387453 -2.299138e+05
     27.0   42.727903  40.809855  ...  11439.129215 -6.174272e+06
     28.0   16.792176  42.163700  ...   5949.228319  4.841186e+04
     29.0   20.750994  24.306281  ... -23322.762200 -7.336071e+04

[3000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
0.0,9.0,15.0,24569817685010.723,47592311509604.195,3071227210626.3403,3399450822114.5854,1.1622140331624546,0.8454834303487635,0.5772582848256183,Reject005=1 : Homoscedasticity
3.0,18.0,6.0,29152605511626.285,21959289749289.402,1714859147742.7227,4391857949857.8804,2.259759225349364,0.16831047456976234,0.9158447627151188,Reject005=1 : Homoscedasticity
4.0,12.0,12.0,84082545117318.52,181185415960368.88,7643867737938.047,16471401450942.625,2.154851708015794,0.198024661438982,0.900987669280509,Reject005=1 : Homoscedasticity
6.0,12.0,12.0,3390728419684.282,39762689379228.71,308248038153.1166,3614789943566.2466,11.726887104373606,0.00015360757938287506,0.9999231962103086,Reject005=0 : Heteroscedasticity
12.0,8.0,16.0,18733954161074.617,62158965578442.65,2676279165867.8022,4143931038562.8433,1.6589921445307168,0.47480919036077873,0.7625954048196106,Reject005=1 : Homoscedasticity
14.0,13.0,11.0,17668491071329.78,66536502281420.06,1472374255944.1484,6653650228142.006,4.450524260012597,0.01300757127818919,0.9934962143609054,Reject005=0 : Heteroscedasticity
15.0,9.0,15.0,27796294794316.945,113728438497637.83,3474536849289.618,8123459892688.416,2.454897805751219,0.1762466641697653,0.9118766679151173,Reject005=1 : Homoscedasticity
16.0,15.0,9.0,59340091140953.945,84096364596212.28,4238577938639.5674,10512045574526.535,2.361988412310022,0.13592237651535877,0.9320388117423206,Reject005=1 : Homoscedasticity
17.0,13.0,11.0,13735403150292.883,17247461062015.754,1144616929191.0735,1724746106201.5754,1.4840018054261996,0.493312518217913,0.7533437408910435,Reject005=1 : Homoscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,9.0,15.0,24569817685010.723,47592311509604.195,3071227210626.3403,3399450822114.5854,1.1622140331624546,1.1545165696512365,Reject005=1 : Homoscedasticity
1.0,13.0,11.0,73639955537968.39,50139695605454.74,6136662961497.366,5013969560545.475,0.804671913018789,0.7266094392506602,Reject005=1 : Homoscedasticity
2.0,12.0,12.0,32861950359214.375,14512856650814.574,2987450032655.852,1319350604619.5068,0.4416310198321878,0.17124341177008506,Reject005=1 : Homoscedasticity
3.0,18.0,6.0,29152605511626.285,21959289749289.402,1714859147742.7227,4391857949857.8804,2.259759225349364,0.16831047456976234,Reject005=1 : Homoscedasticity
4.0,12.0,12.0,84082545117318.52,181185415960368.88,7643867737938.047,16471401450942.625,2.154851708015794,0.198024661438982,Reject005=1 : Homoscedasticity
5.0,13.0,11.0,120787835353582.17,26095795765941.723,10065652946131.848,2609579576594.1724,0.25532774732591673,0.029623539854662033,Reject005=0 : Heteroscedasticity
6.0,12.0,12.0,3390728419684.282,39762689379228.71,308248038153.1166,3614789943566.2466,11.726887104373606,0.00015360757938287506,Reject005=0 : Heteroscedasticity
7.0,13.0,11.0,47042517218078.19,6569599758611.887,3920209768173.182,656959975861.1887,0.16504372854886634,0.00512364802888149,Reject005=0 : Heteroscedasticity
8.0,8.0,16.0,43924456373074.32,6183771398649.601,6274922339010.617,412251426576.64,0.07039098385336251,1.253523187183786e-05,Reject005=0 : Heteroscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    58
Reject005=1 : Homoscedasticity      42
Name: Result_test, dtype: int64